# ファイルをGoogleドライブにアップロード/ダウンロードする場合

学習準備

In [0]:
# googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

# アプリケーションをコピー
!unzip -q /content/drive/My\ Drive/techblog/App.zip -d .

# baselines@tf2の準備
# gymでエラーが出ますが無視してOKです
!pip -q install git+git://github.com/openai/baselines.git@tf2

# ml-agentsの準備
# 公式通りにインストールしようとすると怒られるため、サブディレクトリ機能を使って強引にインストール
!pip -q install "git+git://github.com/Unity-Technologies/ml-agents.git@release_2_branch#subdirectory=ml-agents"
!pip -q install "git+git://github.com/Unity-Technologies/ml-agents.git@release_2_branch#subdirectory=ml-agents-envs"
!pip -q install "git+git://github.com/Unity-Technologies/ml-agents.git@release_2_branch#subdirectory=gym-unity"

学習実行

In [0]:
import tensorflow as tf

from baselines import deepq
from baselines import logger

from mlagents_envs.environment import UnityEnvironment
from gym_unity.envs import UnityToGymWrapper
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel

def main():
    engine_configuration_channel = EngineConfigurationChannel()
    # 時間スケールを20倍に設定
    engine_configuration_channel.set_configuration_parameters(time_scale=20.0)
    unity_env = UnityEnvironment("./App/PushBlock", side_channels=[engine_configuration_channel])
    env = UnityToGymWrapper(unity_env, 0, flatten_branched=True)
    logger.configure('./logs')
    model = deepq.learn(
        env,
        "mlp",
        seed=0,
        lr=2.5e-4,
        total_timesteps=400000,
        buffer_size=50000,
        exploration_fraction=0.05,
        exploration_final_eps=0.1,
        print_freq=20,
        train_freq=5,
        learning_starts=20000,
        target_network_update_freq=50,
        gamma=0.99,
        prioritized_replay=False,
        checkpoint_freq=1000,
        checkpoint_path=None,
        dueling=True
    )

    # モデルを保存
    save_path = "./model"
    ckpt = tf.train.Checkpoint(model=model)
    manager = tf.train.CheckpointManager(ckpt, save_path, max_to_keep=10)
    manager.save()
    # ファイル名に日付を付ける
    import datetime
    filename = 'PushBlock_' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '.zip'
    # 学習したモデルをzip圧縮
    !zip -r $filename model
    # 学習したモデルをGoogleドライブへコピー
    !cp -a $filename /content/drive/My\ Drive/techblog/$filename

if __name__ == '__main__':
    main()


# ファイルを直接アップロード/ダウンロードする場合

In [0]:
# アプリケーションをアップロード
from google.colab import files
files.upload()

# アプリケーションを解凍
!unzip -q App.zip -d .

# baselines@tf2の準備
!pip -q install git+git://github.com/openai/baselines.git@tf2

# ml-agentsの準備
# 公式通りにインストールしようとすると怒られるため、サブディレクトリ機能を使って強引にインストール
!pip -q install "git+git://github.com/Unity-Technologies/ml-agents.git@release_2_branch#subdirectory=ml-agents"
!pip -q install "git+git://github.com/Unity-Technologies/ml-agents.git@release_2_branch#subdirectory=ml-agents-envs"
!pip -q install "git+git://github.com/Unity-Technologies/ml-agents.git@release_2_branch#subdirectory=gym-unity"

In [0]:
import tensorflow as tf

from baselines import deepq
from baselines import logger

from mlagents_envs.environment import UnityEnvironment
from gym_unity.envs import UnityToGymWrapper

def main():
    engine_configuration_channel = EngineConfigurationChannel()
    # 時間スケールを20倍に設定
    engine_configuration_channel.set_configuration_parameters(time_scale=20.0)
    unity_env = UnityEnvironment("./App/PushBlock", side_channels=[engine_configuration_channel])
    env = UnityToGymWrapper(unity_env, 0, flatten_branched=True)
    logger.configure('./logs')
    model = deepq.learn(
        env,
        "mlp",
        lr=2.5e-4,
        total_timesteps=400000,
        buffer_size=50000,
        exploration_fraction=0.05,
        exploration_final_eps=0.1,
        print_freq=20,
        train_freq=5,
        learning_starts=20000,
        target_network_update_freq=50,
        gamma=0.99,
        prioritized_replay=False,
        checkpoint_freq=1000,
        checkpoint_path=None,
        dueling=True
    )

    # モデルを保存
    save_path = "./model"
    ckpt = tf.train.Checkpoint(model=model)
    manager = tf.train.CheckpointManager(ckpt, save_path, max_to_keep=10)
    manager.save()
    # ファイル名に日付を付ける
    import datetime
    filename = 'PushBlock_' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '.zip'
    # 学習したモデルをzip圧縮
    !zip -r $filename model
    # 学習したモデルをダウンロード
    from google.colab import files
    files.download(filename)

if __name__ == '__main__':
    main()

# バッシュコマンドを使い、zip化してGoogleドライブへコピー
%%bash
# ファイル名に日時を付ける
filename=PushBlock_`date "+%Y%m%d_%H%M%S"`.zip
# 学習したモデルをzip圧縮
zip -r $filename model